In [51]:
# ========== セル 1: 環境設定 (ライブラリインストール) ==========
# 目的: このノートブックで必要なPythonライブラリをインストールします。

print("▶️ セル1: 環境設定 を開始します...")
# google-cloud-texttospeech: Google Cloud Text-to-Speech APIを利用するためのライブラリ
# python-dotenv: .envファイルから環境変数を読み込むため (今回はサービスアカウントキー優先なので厳密には不要)
!pip install --upgrade google-cloud-texttospeech python-dotenv
!pip install pydub

print("✅ セル1: 環境設定が完了しました。")

▶️ セル1: 環境設定 を開始します...
✅ セル1: 環境設定が完了しました。


In [65]:
# ========== セル 2: Imports, Constants, Authentication, and Client Setup ==========
print("▶️ セル2: Imports, Constants, Authentication, and Client Setup を開始します...")

# ─── 1. ライブラリのインポート ───
import os
import html
import time
import io
from google.colab import drive
from google.cloud import texttospeech_v1 as texttospeech
import pandas as pd
try:
    from pydub import AudioSegment
except ImportError:
    print("‼️警告: pydubライブラリが見つかりません。"); AudioSegment = None
print("   ✅ 主要ライブラリをインポートしました。")

# ─── 2. Google DriveのマウントとDRIVE_ROOT設定 ───
DRIVE_ROOT = None
try:
    drive.mount('/content/drive', force_remount=True)
    DRIVE_ROOT = "/content/drive/MyDrive/Colab Notebooks/00_Podcast/yt_podcast"
    if not os.path.isdir(DRIVE_ROOT):
        print(f"‼️エラー: DRIVE_ROOTパス不正: {DRIVE_ROOT}")
        DRIVE_ROOT = None
    else:
        print(f"   ✅ Google Drive マウント成功。DRIVE_ROOT: {DRIVE_ROOT}")
except Exception as e:
    print(f"‼️エラー: Driveマウント失敗: {e}")
    DRIVE_ROOT = None

# ─── 3. サービスアカウントキーJSONファイルの設定と認証 ───
SERVICE_ACCOUNT_KEY_FILENAME = "gen-lang-client-0377684772-09447c306d22.json"
TTS_CLIENT_INITIALIZED = False
tts_client = None
if DRIVE_ROOT:
    key_directory = os.path.join(DRIVE_ROOT, "Key")
    SERVICE_ACCOUNT_KEY_PATH = os.path.join(key_directory, SERVICE_ACCOUNT_KEY_FILENAME)
    if os.path.exists(SERVICE_ACCOUNT_KEY_PATH):
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = SERVICE_ACCOUNT_KEY_PATH
        print(f"   ✅ 環境変数 GOOGLE_APPLICATION_CREDENTIALS にSAキー設定: {SERVICE_ACCOUNT_KEY_PATH}")
        try:
            tts_client = texttospeech.TextToSpeechClient()
            print("   ✅ Text-to-Speech クライアント初期化成功。")
            TTS_CLIENT_INITIALIZED = True
        except Exception as e:
            print(f"‼️エラー: TTSクライアント初期化失敗: {e}")
    else:
        print(f"‼️エラー: SAキーファイルが見つかりません: {SERVICE_ACCOUNT_KEY_PATH}")
else:
    print("‼️エラー: DRIVE_ROOT が無効なため、SAキーパス設定不可。")

# ─── 4. 関連ディレクトリパスの定義・確認 ───
if DRIVE_ROOT:
    SCRIPTS_DIR_TTS = os.path.join(DRIVE_ROOT, "scripts")
    OUTPUTS_DIR_TTS = os.path.join(DRIVE_ROOT, "outputs")
    os.makedirs(OUTPUTS_DIR_TTS, exist_ok=True)
    print(f"   ℹ️ 台本入力ディレクトリ: {SCRIPTS_DIR_TTS}")
    print(f"   ℹ️ 音声出力ディレクトリ: {OUTPUTS_DIR_TTS}")
else:
    SCRIPTS_DIR_TTS = OUTPUTS_DIR_TTS = None

# ─── 5. キャラクターボイス設定 ───
VOICE_SETTINGS = {
    'Maya': texttospeech.VoiceSelectionParams(
        language_code='ja-JP',
        name='ja-JP-Neural2-B'    # Mayaはこれまで通り
    ),
    'Shiba': texttospeech.VoiceSelectionParams(
        language_code='ja-JP',
        name='ja-JP-Neural2-C'    # 存在が確認できたCに変更
    ),
    'Default': texttospeech.VoiceSelectionParams(
        language_code='ja-JP'     # nameを外してlanguage_codeのみ
    )
}
print("   ✅ キャラクターボイス設定を定義しました。")

# ─── 6. オーディオ設定 ───
AUDIO_CONFIG_MP3 = texttospeech.AudioConfig(
    audio_encoding=texttospeech.AudioEncoding.MP3,
    speaking_rate=1.0,
    pitch=0.0
)
print("   ✅ オーディオ設定 (MP3) を定義しました。")

print("✅ セル2: Imports, Constants, Authentication, and Client Setup が完了しました。")


▶️ セル2: Imports, Constants, Authentication, and Client Setup を開始します...
   ✅ 主要ライブラリをインポートしました。
Mounted at /content/drive
   ✅ Google Drive マウント成功。DRIVE_ROOT: /content/drive/MyDrive/Colab Notebooks/00_Podcast/yt_podcast
   ✅ 環境変数 GOOGLE_APPLICATION_CREDENTIALS にSAキー設定: /content/drive/MyDrive/Colab Notebooks/00_Podcast/yt_podcast/Key/gen-lang-client-0377684772-09447c306d22.json
   ✅ Text-to-Speech クライアント初期化成功。
   ℹ️ 台本入力ディレクトリ: /content/drive/MyDrive/Colab Notebooks/00_Podcast/yt_podcast/scripts
   ℹ️ 音声出力ディレクトリ: /content/drive/MyDrive/Colab Notebooks/00_Podcast/yt_podcast/outputs
   ✅ キャラクターボイス設定を定義しました。
   ✅ オーディオ設定 (MP3) を定義しました。
✅ セル2: Imports, Constants, Authentication, and Client Setup が完了しました。


In [66]:
# ========== セル 3: STEP5 テスト実行 (スピーカー別 API 呼び出し方式) ==========
print("▶️ セル3: スピーカー別の音声合成テストを開始します...")

from google.cloud import texttospeech_v1 as texttospeech
import io
import os
from pydub import AudioSegment

# --- ヘルパー関数: テキスト＋VoiceSelectionParams で合成 ---
def synthesize_text_for_tts(text, voice_params, audio_config, client):
    # テキスト入力を使うことで SSML の <voice> タグ不要に
    synthesis_input = texttospeech.SynthesisInput(text=text)
    response = client.synthesize_speech(
        request={
            "input": synthesis_input,
            "voice": voice_params,
            "audio_config": audio_config,
        }
    )
    return response.audio_content

# --- テスト用: 最初の台本ファイルを読み込み ---
script_files = sorted([f for f in os.listdir(SCRIPTS_DIR_TTS) if f.endswith('.txt')])
if not script_files:
    print("‼️ テスト用の台本ファイルが見つかりません。セル2でパスを確認してください。")
else:
    src = script_files[0]
    vid = src[:-4]
    print(f"   ℹ️ テスト対象: {src} (Video ID: {vid})")
    lines = []
    with open(os.path.join(SCRIPTS_DIR_TTS, src), encoding='utf-8') as f:
        for raw in f:
            line = raw.strip()
            if line:
                lines.append(line)

    # --- 台本を行ごとに "(テキスト, VoiceSelectionParams)" のリストに変換 ---
    segments = []
    current_speaker = 'Default'
    for line in lines:
        if ':' in line:
            tag, content = line.split(':', 1)
            tag = tag.strip()
            if tag in VOICE_SETTINGS:
                current_speaker = tag
                text = content.strip()
            else:
                text = line
        else:
            text = line
        # 話者タグに対応する voice パラメータを取得
        voice_params = VOICE_SETTINGS.get(current_speaker, VOICE_SETTINGS['Default'])
        segments.append((text, voice_params))

    # --- 各セグメントを合成して結合 ---
    audio_list = []
    for idx, (txt, vp) in enumerate(segments, 1):
        print(f"   💬 セグメント {idx}: voice={vp.name}, text='{txt[:20]}…'")
        audio_bytes = synthesize_text_for_tts(txt, vp, AUDIO_CONFIG_MP3, tts_client)
        audio_list.append(AudioSegment.from_file(io.BytesIO(audio_bytes), format='mp3'))

    print(f"   🔗 {len(audio_list)} 個のチャンクを結合中...")
    combined = AudioSegment.empty()
    for seg in audio_list:
        combined += seg

    # --- 出力 ---
    out_fn = f"{vid}_speaker_test.mp3"
    out_path = os.path.join(OUTPUTS_DIR_TTS, out_fn)
    combined.export(out_path, format='mp3')
    print(f"✅ 合成完了: {out_path}")

print("✅ セル3: テスト実行 完了しました。")


▶️ セル3: スピーカー別の音声合成テストを開始します...
   ℹ️ テスト対象: 433nP6kGRbQ.txt (Video ID: 433nP6kGRbQ)
   💬 セグメント 1: voice=ja-JP-Neural2-B, text='こんにちは、マヤです。…'
   💬 セグメント 2: voice=ja-JP-Neural2-C, text='こんにちは、シバわん！…'
   💬 セグメント 3: voice=ja-JP-Neural2-B, text='これはテスト音声です。…'
   💬 セグメント 4: voice=ja-JP-Neural2-C, text='うまくいくかな？わくわく！…'
   🔗 4 個のチャンクを結合中...
✅ 合成完了: /content/drive/MyDrive/Colab Notebooks/00_Podcast/yt_podcast/outputs/433nP6kGRbQ_speaker_test.mp3
✅ セル3: テスト実行 完了しました。


In [ ]:
# ========== セル 4: STEP5 本番実行 (全ファイルの音声合成＆出力) ==========
print("▶️ セル4: 本番実行を開始します...")

import os
import io
from pydub import AudioSegment
from google.cloud import texttospeech_v1 as texttospeech

# --- ヘルパー関数はセル3と同じものを利用 ---
def synthesize_text_for_tts(text, voice_params, audio_config, client):
    synthesis_input = texttospeech.SynthesisInput(text=text)
    response = client.synthesize_speech(
        request={
            "input": synthesis_input,
            "voice": voice_params,
            "audio_config": audio_config,
        }
    )
    return response.audio_content

# 台本ディレクトリの全 .txt を処理
script_files = sorted(f for f in os.listdir(SCRIPTS_DIR_TTS) if f.endswith('.txt'))
if not script_files:
    print("‼️ 本番用の台本ファイルが見つかりません。セル2でパスを確認してください。")
else:
    for src in script_files:
        vid = src[:-4]
        out_path = os.path.join(OUTPUTS_DIR_TTS, f"{vid}.mp3")
        # 既存ファイルはスキップ
        if os.path.exists(out_path):
            print(f"   ⏭️ スキップ: {out_path} は既に存在します。")
            continue

        print(f"\n   ▶️ 処理中: {src} → {vid}.mp3")
        # ファイル読み込み
        with open(os.path.join(SCRIPTS_DIR_TTS, src), 'r', encoding='utf-8') as f:
            lines = [ln.strip() for ln in f if ln.strip()]

        # 行ごとに (テキスト, VoiceSelectionParams) のリスト化
        segments = []
        current_speaker = 'Default'
        for line in lines:
            if ':' in line:
                tag, content = line.split(':', 1)
                tag = tag.strip()
                if tag in VOICE_SETTINGS:
                    current_speaker = tag
                    text = content.strip()
                else:
                    text = line
            else:
                text = line
            voice_params = VOICE_SETTINGS.get(current_speaker, VOICE_SETTINGS['Default'])
            segments.append((text, voice_params))

        # 各セグメントを合成＆結合
        audio_list = []
        for idx, (txt, vp) in enumerate(segments, 1):
            print(f"      🔉 セグメント{idx}: voice={vp.name if vp.name else vp.language_code}, text='{txt[:20]}…'")
            audio_bytes = synthesize_text_for_tts(txt, vp, AUDIO_CONFIG_MP3, tts_client)
            audio_list.append(AudioSegment.from_file(io.BytesIO(audio_bytes), format='mp3'))

        print(f"      🔗 {len(audio_list)} チャンクを結合中...")
        combined = AudioSegment.empty()
        for seg in audio_list:
            combined += seg

        # MP3 出力
        combined.export(out_path, format='mp3')
        print(f"   ✅ 出力完了: {out_path}")

print("\n✅ セル4: 本番実行 完了しました。")


▶️ セル4: 本番実行を開始します...

   ▶️ 処理中: 433nP6kGRbQ.txt → 433nP6kGRbQ.mp3
      🔉 セグメント1: voice=ja-JP-Neural2-B, text='こんにちは、マヤです。…'
      🔉 セグメント2: voice=ja-JP-Neural2-C, text='こんにちは、シバわん！…'
      🔉 セグメント3: voice=ja-JP-Neural2-B, text='これはテスト音声です。…'
      🔉 セグメント4: voice=ja-JP-Neural2-C, text='うまくいくかな？わくわく！…'
      🔗 4 チャンクを結合中...
   ✅ 出力完了: /content/drive/MyDrive/Colab Notebooks/00_Podcast/yt_podcast/outputs/433nP6kGRbQ.mp3

   ▶️ 処理中: G3fjxgYCct0.txt → G3fjxgYCct0.mp3
      🔉 セグメント1: voice=ja-JP, text='```…'
      🔉 セグメント2: voice=ja-JP-Neural2-B, text='皆さん、こんにちは！今日のポッドキャスト…'
      🔉 セグメント3: voice=ja-JP-Neural2-C, text='わーい！Mayaお姉ちゃん、今日はどんな…'
      🔉 セグメント4: voice=ja-JP-Neural2-B, text='大丈夫ですよ、Shibaちゃん。難しい言…'
      🔉 セグメント5: voice=ja-JP-Neural2-C, text='うん！…'
      🔉 セグメント6: voice=ja-JP-Neural2-B, text='まず一つ目は、「マーケティングは、量より…'
      🔉 セグメント7: voice=ja-JP-Neural2-C, text='それってどういうこと？たくさん宣伝した方…'
      🔉 セグメント8: voice=ja-JP-Neural2-B, text='確かに、量をこなすことも重要ですが、それ…'
      🔉 セグメント9: voice=ja-JP-Ne

KeyboardInterrupt: 